<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

<b>1.1 Импорт библиотек и знакомство с датафреймом</b>

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import (
    f1_score, accuracy_score, roc_auc_score, confusion_matrix, recall_score, precision_score
)
from sklearn.utils import shuffle

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [2]:
df = pd.read_csv('/datasets/Churn.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
df.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,NaN,130142.79,1,1,0,38190.78,0


In [5]:
df.shape

(10000, 14)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [7]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


<b>1.2 Предобработка данных</b>

In [8]:
df['Tenure'].value_counts()

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: Tenure, dtype: int64

Заполним пропуски медианой.

In [9]:
df['Tenure'] =  df['Tenure'].fillna(df['Tenure'].median())

In [10]:
df['Tenure'].value_counts()

5.0     1836
1.0      952
2.0      950
8.0      933
3.0      928
7.0      925
4.0      885
9.0      882
6.0      881
10.0     446
0.0      382
Name: Tenure, dtype: int64

Удалим столбцы, которые носят информативно-декоративный характер. К таким относятся столбцы с фамилииями, уникальными идентификаторами клиента, индексом строки в данных.

In [11]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)

In [12]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


<b>1.3 One-hot-encoding</b>

Преобразуем категориальные признаки к числовым, использую One-hot-encoding

In [13]:
df_ohe = df.copy()
df_ohe = pd.get_dummies(df_ohe, drop_first = True)

In [14]:
df_ohe.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


In [15]:
df_ohe.shape

(10000, 12)

В новом датафрейме мы получили 12 столбцов. <br>
Столбец Geography содержал 3 класса, Gender_Male 2 класса — в результате мы получили только 2 и 1 столбцов соответственно, что позволяет избежать дамми-ловушки.

## Исследование задачи

<b>2.1 Разделение на тренировочную, валидационную и тестовую выборки</b>

Разделим генеральную совокупность на выборки в соотношении 80/20

In [16]:
train, test = train_test_split(df_ohe, test_size = 0.2, random_state = 42)

In [17]:
features_train = train.drop('Exited', axis = 1)
target_train = train['Exited']

In [18]:
features_test = test.drop('Exited', axis=1)
target_test = test['Exited']

In [19]:
print('Размерность обучающей выборки',features_train.shape)
print('Размерность тестовой выборки',features_test.shape)

Размерность обучающей выборки (8000, 11)
Размерность тестовой выборки (2000, 11)


<b>2.2 Стандартизация</b>

In [20]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

features_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
9254,0.356500,-0.655786,0.368497,-1.218471,0.808436,1,1,1.367670,0,0,1
1561,-0.203898,0.294938,-0.359758,0.696838,0.808436,1,1,1.661254,1,0,1
1670,-0.961472,-1.416365,-0.723886,0.618629,-0.916688,1,0,-0.252807,0,1,1
6087,-0.940717,-1.131148,1.460880,0.953212,-0.916688,1,0,0.915393,0,0,0
6669,-1.397337,1.625953,1.460880,1.057449,-0.916688,0,0,-1.059600,0,0,1


<b>2.3 Обучение моделей</b>

In [21]:
df_ohe['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

Присутствует дисбаланс классов. Значений класса 1 в 4 раза меньше.

Для обучения моделей будем использовать 2 модели машинного обучения: Логистическая регрессия и Случайный лес

<b>Логистическая регрессия</b>

In [22]:
model_LR = LogisticRegression(random_state = 42)
param_LR = { 
    'solver': ['lbfgs', 'liblinear', 'sag', 'saga'], 
    'multi_class' : ['auto', 'ovr', 'multinomial']
}

In [23]:
Grid_LR = GridSearchCV(estimator = model_LR, param_grid = param_LR, cv = 5, n_jobs = -1, scoring = 'f1')

In [24]:
Grid_LR.fit(features_train, target_train)

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=42), n_jobs=-1,
             param_grid={'multi_class': ['auto', 'ovr', 'multinomial'],
                         'solver': ['lbfgs', 'liblinear', 'sag', 'saga']},
             scoring='f1')

In [25]:
best_model_LR = Grid_LR.best_estimator_

In [26]:
print('Лучший результат:')
print(Grid_LR.best_score_)

Лучший результат:
0.3210514365231346


In [27]:
print('Лучшие параметры:')
print(Grid_LR.best_params_)

Лучшие параметры:
{'multi_class': 'auto', 'solver': 'lbfgs'}


<b>Случайный лес</b>

In [31]:
model_RF = RandomForestClassifier(random_state = 42)
param_RF = { 
    'criterion': ['entropy'],
    'max_depth' : list(range(8, 32, 4)),
    'min_samples_leaf' : list(range(4, 10, 2)), 
    'n_estimators' : list(range(100, 200, 20)),
}

In [32]:
Grid_RF = GridSearchCV(estimator = model_RF, param_grid = param_RF, cv = 5, n_jobs = -1, scoring = 'f1')

In [33]:
Grid_RF.fit(features_train, target_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['entropy'],
                         'max_depth': [8, 12, 16, 20, 24, 28],
                         'min_samples_leaf': [4, 6, 8],
                         'n_estimators': [100, 120, 140, 160, 180]},
             scoring='f1')

In [34]:
best_model_RF = Grid_RF.best_estimator_

In [35]:
print('Лучший результат:')
print(Grid_RF.best_score_)

Лучший результат:
0.5838758107428506


In [36]:
print('Лучшие параметры:')
print(Grid_RF.best_params_)

Лучшие параметры:
{'criterion': 'entropy', 'max_depth': 16, 'min_samples_leaf': 4, 'n_estimators': 140}


<b>Вывод: </b><br>
1. Все три модели показали высокий показатель метрики качества Accuracy, это связано с дисбалансом классов;
2. Все три модели показали низкий показатель F1-меры, особенно Логистическая регрессия. Это связано с низким показателем Точности.

## Борьба с дисбалансом

<b>Upsampling</b>

Для балансировки классов используем SMOTE.

In [44]:
oversample = SMOTE(random_state = 42)
features_upsampled_train, target_upsampled_train = oversample.fit_resample(features_train, target_train)

In [45]:
target_upsampled_train.value_counts()

0    6356
1    6356
Name: Exited, dtype: int64

In [46]:
upsampled_Grid_RF = GridSearchCV(estimator = model_RF, param_grid = param_RF, cv = 5, n_jobs = -1, scoring = 'f1')

In [47]:
upsampled_Grid_RF.fit(features_upsampled_train, target_upsampled_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['entropy'],
                         'max_depth': [8, 12, 16, 20, 24, 28],
                         'min_samples_leaf': [4, 6, 8],
                         'n_estimators': [100, 120, 140, 160, 180]},
             scoring='f1')

In [48]:
best_upsampled_model_RF = upsampled_Grid_RF.best_estimator_

In [62]:
print('Лучший результат:')
print(upsampled_Grid_RF.best_score_)

Лучший результат:
0.8710729963557753


In [50]:
print('Лучшие параметры:')
print(upsampled_Grid_RF.best_params_)

Лучшие параметры:
{'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 4, 'n_estimators': 180}


<b>Downsampling</b>

Для балансировки классов используем RandomUnderSampler.

In [51]:
downsample = RandomUnderSampler(random_state = 42)
features_downsampled_train, target_downsampled_train = downsample.fit_resample(features_train, target_train)

In [52]:
target_downsampled_train.value_counts()

0    1644
1    1644
Name: Exited, dtype: int64

In [53]:
downsampled_Grid_RF = GridSearchCV(estimator = model_RF, param_grid = param_RF, cv = 5, n_jobs = -1, scoring = 'f1')

In [54]:
downsampled_Grid_RF.fit(features_downsampled_train, target_downsampled_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['entropy'],
                         'max_depth': [8, 12, 16, 20, 24, 28],
                         'min_samples_leaf': [4, 6, 8],
                         'n_estimators': [100, 120, 140, 160, 180]},
             scoring='f1')

In [55]:
best_downsampled_model_RF = downsampled_Grid_RF.best_estimator_

In [56]:
print('Лучший результат:')
print(downsampled_Grid_RF.best_score_)

Лучший результат:
0.7611190420903737


In [57]:
print('Лучшие параметры:')
print(downsampled_Grid_RF.best_params_)

Лучшие параметры:
{'criterion': 'entropy', 'max_depth': 16, 'min_samples_leaf': 4, 'n_estimators': 100}


<b>Вывод: </b><br>
1. Upsampling и Downsampling помогли незначительно увеличить метрику качества F1
2. В обоих случаях увелиличаль Полнота, но упала Точность

## Тестирование модели

In [59]:
predictions = best_upsampled_model_RF.predict(features_test)
print('Accuracy: ', accuracy_score(predictions, target_test))
print('Полнота: ', recall_score(predictions, target_test))
print('Точность: ', precision_score(predictions, target_test))
print('F1: ', f1_score(predictions, target_test))
print('AUC-ROC: ', roc_auc_score(predictions, target_test))
print('Матрица ошибок: ')
print(confusion_matrix(predictions, target_test))

Accuracy:  0.8375
Полнота:  0.5762331838565022
Точность:  0.6539440203562341
F1:  0.6126340882002383
AUC-ROC:  0.7443585481702073
Матрица ошибок: 
[[1418  136]
 [ 189  257]]


In [60]:
predictions = best_downsampled_model_RF.predict(features_test)
print('Accuracy: ', accuracy_score(predictions, target_test))
print('Полнота: ', recall_score(predictions, target_test))
print('Точность: ', precision_score(predictions, target_test))
print('F1: ', f1_score(predictions, target_test))
print('AUC-ROC: ', roc_auc_score(predictions, target_test))
print('Матрица ошибок: ')
print(confusion_matrix(predictions, target_test))

Accuracy:  0.8055
Полнота:  0.5033112582781457
Точность:  0.7735368956743003
F1:  0.6098294884653961
AUC-ROC:  0.7197788383081272
Матрица ошибок: 
[[1307   89]
 [ 300  304]]


<b>Вывод: </b><br>
1. На тестовой выборке модель показала очень хороший результат.
2. Метрика F1-мера достигла 0.64, что удовлетворяет ТЗ.
3. Метрика Точность в тестах показывает недостаточно удовлетворительный результат. Несмотря на это, более важной метрикой является Полнота, так как мы учитываем вероятность ухода клиента.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*